In [1]:
!pip install -q requests beautifulsoup4

In [2]:
import requests,urllib
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
import pandas as pd


## data source : https://www.nseindia.com/products-services/indices-niftysmallcap250-index

In [3]:
df = pd.read_csv("/kaggle/input/nifty-smallcap-250/ind_niftysmallcap250list.csv")
df.tail(5)

,Company Name,Industry,Symbol,Series,ISIN Code
245,Welspun India Ltd.,Textiles,WELSPUNIND,EQ,INE192B01031
246,Westlife Development Ltd.,Consumer Services,WESTLIFE,EQ,INE274F01020
247,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027
248,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010
249,eClerx Services Ltd.,Services,ECLERX,EQ,INE738I01010


## data source : https://www.screener.in/company/RELIANCE/consolidated/#documents

In [4]:

def annual_report_links(ticker):
    url = 'https://www.screener.in/company/'+ticker+'/consolidated/#documents'
    response = requests.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    # <h3>Annual reports</h3>
    reports_header = soup.find('h3', text='Annual reports')
    # <div class= "show-more-box highlight">
    #    <ul class= "list-links">
    #         <li>
    #             <a href="link to documents"></a>
    #         </li>
    #         ....
    #    </ul>
    #</div>
    # find all the <a> tags inside the <div> element with class "show-more-box" under the <h3> tag
    reports_links = reports_header.find_next_sibling('div',
                                                     class_='show-more-box').find_all('a')
    reports_list = []
    for link in tqdm(reports_links):
        href_link = link.get("href")
        if href_link.split("/")[-3:]!="zip":
            reports_list.append(href_link)
            
    return reports_list


In [5]:
# reports_list= annual_report_links("ECLERX")
# reports_list

In [6]:

def generate_report_link(data):
    
    fin_2022, fin_2021, fin_2020, fin_2019, fin_2018 = [], [], [], [], []

    for ticker in tqdm(data["Symbol"]):
        reports_list = annual_report_links(ticker)
        fin_2022.append(reports_list[0] if len(reports_list) > 0 else "")
        fin_2021.append(reports_list[1] if len(reports_list) > 1 else "")
        fin_2020.append(reports_list[2] if len(reports_list) > 2 else "")
        fin_2019.append(reports_list[3] if len(reports_list) > 3 else "")
        fin_2018.append(reports_list[4] if len(reports_list) > 4 else "")

    data["Year_2022"] = np.array(fin_2022)
    data["Year_2021"] = np.array(fin_2021)
    data["Year_2020"] = np.array(fin_2020)
    data["Year_2019"] = np.array(fin_2019)
    data["Year_2018"] = np.array(fin_2018)
    
    return data


data = df.copy(deep=True)
data = data.tail(10).reset_index(drop=True)
data = generate_report_link(data)
data.head(10)

100%|██████████| 10/10 [00:08<00:00,  1.13it/s]


,Company Name,Industry,Symbol,Series,ISIN Code,Year_2022,Year_2021,Year_2020,Year_2019,Year_2018
0,Vaibhav Global Ltd.,Consumer Durables,VAIBHAVGBL,EQ,INE884A01027,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
1,Vardhman Textiles Ltd.,Textiles,VTL,EQ,INE825A01020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
2,Varroc Engineering Ltd.,Automobile and Auto Components,VARROC,EQ,INE665L01035,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
3,Vijaya Diagnostic Centre Ltd.,Healthcare,VIJAYA,EQ,INE043W01024,https://archives.nseindia.com/annual_reports/A...,,,,
4,Welspun Corp Ltd.,Capital Goods,WELCORP,EQ,INE191B01025,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
5,Welspun India Ltd.,Textiles,WELSPUNIND,EQ,INE192B01031,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
6,Westlife Development Ltd.,Consumer Services,WESTLIFE,EQ,INE274F01020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
7,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
8,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...
9,eClerx Services Ltd.,Services,ECLERX,EQ,INE738I01010,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...


In [7]:
import shutil,os

if os.path.exists("content"):
    shutil.rmtree("content/")
else:
    os.makedirs("content")

In [8]:

def download_file(data):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0"}

    paths = {"2022": [], "2021": [], "2020": [], "2019": [], "2018": []}

    for _, row in tqdm(data.iterrows(), total=data.shape[0]):
        company = row[2]
        for i in range(5):
            year = str(2022 - i)
            src = row[5+i]
            if src:
                dst = f"content/{company}_{year}.pdf"
                req = urllib.request.Request(src, headers=headers)
                with urllib.request.urlopen(req) as response, open(dst, "wb") as f:
                    f.write(response.read())
                paths[year].append(dst)
            else:
                paths[year].append("")

    for year, path in paths.items():
        data[f"Path_{year}"] = np.array(path)


In [9]:
data2 = data.copy(deep=True)
download_file(data2)

100%|██████████| 10/10 [00:35<00:00,  3.54s/it]


In [10]:
data2.head(10)

,Company Name,Industry,Symbol,Series,ISIN Code,Year_2022,Year_2021,Year_2020,Year_2019,Year_2018,Path_2022,Path_2021,Path_2020,Path_2019,Path_2018
0,Vaibhav Global Ltd.,Consumer Durables,VAIBHAVGBL,EQ,INE884A01027,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/VAIBHAVGBL_2022.pdf,content/VAIBHAVGBL_2021.pdf,content/VAIBHAVGBL_2020.pdf,content/VAIBHAVGBL_2019.pdf,content/VAIBHAVGBL_2018.pdf
1,Vardhman Textiles Ltd.,Textiles,VTL,EQ,INE825A01020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/VTL_2022.pdf,content/VTL_2021.pdf,content/VTL_2020.pdf,content/VTL_2019.pdf,content/VTL_2018.pdf
2,Varroc Engineering Ltd.,Automobile and Auto Components,VARROC,EQ,INE665L01035,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/VARROC_2022.pdf,content/VARROC_2021.pdf,content/VARROC_2020.pdf,content/VARROC_2019.pdf,content/VARROC_2018.pdf
3,Vijaya Diagnostic Centre Ltd.,Healthcare,VIJAYA,EQ,INE043W01024,https://archives.nseindia.com/annual_reports/A...,,,,,content/VIJAYA_2022.pdf,,,,
4,Welspun Corp Ltd.,Capital Goods,WELCORP,EQ,INE191B01025,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/WELCORP_2022.pdf,content/WELCORP_2021.pdf,content/WELCORP_2020.pdf,content/WELCORP_2019.pdf,content/WELCORP_2018.pdf
5,Welspun India Ltd.,Textiles,WELSPUNIND,EQ,INE192B01031,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/WELSPUNIND_2022.pdf,content/WELSPUNIND_2021.pdf,content/WELSPUNIND_2020.pdf,content/WELSPUNIND_2019.pdf,content/WELSPUNIND_2018.pdf
6,Westlife Development Ltd.,Consumer Services,WESTLIFE,EQ,INE274F01020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/WESTLIFE_2022.pdf,content/WESTLIFE_2021.pdf,content/WESTLIFE_2020.pdf,content/WESTLIFE_2019.pdf,content/WESTLIFE_2018.pdf
7,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/ZENSARTECH_2022.pdf,content/ZENSARTECH_2021.pdf,content/ZENSARTECH_2020.pdf,content/ZENSARTECH_2019.pdf,content/ZENSARTECH_2018.pdf
8,Zydus Wellness Ltd.,Fast Moving Consumer Goods,ZYDUSWELL,EQ,INE768C01010,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/ZYDUSWELL_2022.pdf,content/ZYDUSWELL_2021.pdf,content/ZYDUSWELL_2020.pdf,content/ZYDUSWELL_2019.pdf,content/ZYDUSWELL_2018.pdf
9,eClerx Services Ltd.,Services,ECLERX,EQ,INE738I01010,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bs

In [11]:
!pip install -q PyPDF2 pdfminer.six 


In [12]:
# from io import StringIO
# from pdfminer.high_level import extract_text_to_fp
# from pdfminer.layout import LAParams
# import re

# def extract_text_from_pdf(filepath):
#     with open(filepath, 'rb') as fp:
#         output = StringIO()
#         extract_text_to_fp(fp, output, laparams=LAParams())
#         text = output.getvalue()
#     return text


# def extract_mda(text):
#     mda_keywords = ['management discussion and analysis', 'managements discussion and analysis', 'md&a']
#     for keyword in tqdm(mda_keywords):
#         pattern = re.compile(keyword, re.IGNORECASE)
#         match = pattern.search(text)
#         if match:
#             start = match.start()
#             end = text.find('\n\n', start)
#             if end == -1:
#                 end = None
#             return text[start:end]
#     return None

# mda_texts = []
# filepaths = ["/kaggle/working/content/ECLERX_2022.pdf"]
# for filepath in filepaths:
#     text = extract_text_from_pdf(filepath)
#     mda = extract_mda(text)
#     mda_texts.append(mda)


# print(mda_texts)    


In [13]:
#filepath = '/kaggle/working/content/ECLERX_2022.pdf'

In [14]:
from PyPDF2 import PdfReader
from io import StringIO


def extract_mdna_from_pdf(filepath):
    mdna_keywords = ['management discussion and analysis', 'managements discussion and analysis', 'md&a']

    with open(filepath, 'rb') as file:
        pdf = PdfReader(file)
        mdna_text = ""
        for page in tqdm(range(len(pdf.pages))):
            text = pdf.pages[page].extract_text()
            for keyword in mdna_keywords:
                if keyword.lower() in text.lower():
                    index = text.lower().index(keyword.lower())
                    mdna_text += text[index:]

        return mdna_text


def get_entire_mda_text(data):
    mdnas = {"2022": [], "2021": [], "2020": [], "2019": [], "2018": []}
    for _, row in tqdm(data.iterrows(), total=data.shape[0]):
        company = row[2]
        for i in range(5):
            year = str(2022 - i)
            src = row[10+i]
            if src:
                try:
                    mdna_text = extract_mdna_from_pdf(src)
                except:
                    print(f"Error reading file: {src}")
                    mdna_text = ""
                if mdna_text:
                    mdnas[year].append(mdna_text)
                else:
                    mdnas[year].append("")
            else:
                mdnas[year].append("")
    for year, mdna in mdnas.items():
        data[f"MDnA_{year}"] = np.array(mdna)




In [15]:
data3 = data2.copy(deep=True)
get_entire_mda_text(data3)


100%|██████████| 124/124 [00:31<00:00,  3.94it/s]

100%|██████████| 105/105 [00:20<00:00,  5.15it/s]

100%|██████████| 221/221 [00:09<00:00, 23.08it/s]

100%|██████████| 161/161 [00:11<00:00, 14.13it/s]

100%|██████████| 282/282 [00:15<00:00, 18.78it/s]

100%|██████████| 271/271 [00:10<00:00, 24.91it/s]

100%|██████████| 274/274 [00:10<00:00, 25.41it/s]

100%|██████████| 288/288 [00:14<00:00, 19.74it/s]

100%|██████████| 183/183 [00:29<00:00,  6.25it/s]

100%|██████████| 165/165 [00:31<00:00,  5.28it/s]

100%|██████████| 309/309 [00:47<00:00,  6.49it/s]

100%|██████████| 254/254 [00:17<00:00, 14.37it/s]

 40%|████      | 4/10 [05:12<06:54, 69.14s/it] 

Error reading file: content/VIJAYA_2022.pdf



100%|██████████| 432/432 [00:31<00:00, 13.68it/s]

100%|██████████| 363/363 [00:35<00:00, 10.17it/s]

100%|██████████| 348/348 [00:44<00:00,  7.86it/s]

100%|██████████| 317/317 [00:25<00:00, 12.23it/s]

100%|██████████| 210/210 [00:30<00:00,  6.94it/s]

100%|██████████| 323/323 [00:31<00:00, 10.15it/s]

100%|██████████| 330/330 [00:31<00:00, 10.43it/s]

100%|██████████| 287/287 [00:19<00:00, 15.02it/s]

100%|██████████| 208/208 [00:18<00:00, 11.16it/s]

100%|██████████| 208/208 [00:14<00:00, 14.82it/s]

100%|██████████| 186/186 [00:12<00:00, 14.80it/s]

100%|██████████| 168/168 [00:06<00:00, 26.46it/s]

100%|██████████| 360/360 [00:19<00:00, 18.68it/s]

100%|██████████| 309/309 [00:19<00:00, 15.74it/s]

100%|██████████| 280/280 [00:13<00:00, 20.86it/s]

100%|██████████| 313/313 [00:12<00:00, 24.48it/s]

100%|██████████| 217/217 [00:11<00:00, 18.22it/s]

100%|██████████| 204/204 [00:16<00:00, 12.32it/s]

100%|██████████| 193/193 [00:12<00:00, 15.95it/s]

100%|██████████| 213/213 [00:0

In [16]:
data3.head(10)

,Company Name,Industry,Symbol,Series,ISIN Code,Year_2022,Year_2021,Year_2020,Year_2019,Year_2018,Path_2022,Path_2021,Path_2020,Path_2019,Path_2018,MDnA_2022,MDnA_2021,MDnA_2020,MDnA_2019,MDnA_2018
0,Vaibhav Global Ltd.,Consumer Durables,VAIBHAVGBL,EQ,INE884A01027,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/VAIBHAVGBL_2022.pdf,content/VAIBHAVGBL_2021.pdf,content/VAIBHAVGBL_2020.pdf,content/VAIBHAVGBL_2019.pdf,content/VAIBHAVGBL_2018.pdf,"Management Discussion and Analysis, \ndescribi...","Management Discussion and Analysis \nLearning,...",Management Discussion and Analysis .............,"Management Discussion and Analysis Report, whi...","Management Discussion and Analysis Report, whi..."
1,Vardhman Textiles Ltd.,Textiles,VTL,EQ,INE825A01020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/VTL_2022.pdf,content/VTL_2021.pdf,content/VTL_2020.pdf,content/VTL_2019.pdf,content/VTL_2018.pdf,Management Discussion and Analysis\nGlobal eco...,Management Discussion and Analysis Report for ...,Management Discussion and Analysis 24\nBusines...,Management Discussion and Analysis\nGlobal Eco...,Management Discussion and Analysis Report\n38 ...
2,Varroc Engineering Ltd.,Automobile and Auto Components,VARROC,EQ,INE665L01035,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/VARROC_2022.pdf,content/VARROC_2021.pdf,content/VARROC_2020.pdf,content/VARROC_2019.pdf,content/VARROC_2018.pdf,Management Discussion and Analysis\n1. ECONOM...,Management Discussion and Analysis \n46 Board...,Management Discussion and Analysis \n1. Econom...,Management Discussion and Analysis\nECONOMIC R...,Management Discussion and Analysis report. \n3...
3,Vijaya Diagnostic Centre Ltd.,Healthcare,VIJAYA,EQ,INE043W01024,https://archives.nseindia.com/annual_reports/A...,,,,,content/VIJAYA_2022.pdf,,,,,,,,,
4,Welspun Corp Ltd.,Capital Goods,WELCORP,EQ,INE191B01025,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/WELCORP_2022.pdf,content/WELCORP_2021.pdf,content/WELCORP_2020.pdf,content/WELCORP_2019.pdf,content/WELCORP_2018.pdf,Management Discussion and Analysis 42\nDirecto...,Management Discussion and Analysis\n69 Directo...,MD&A) should be \nread in c onjunction with th...,Management Discussion and Analysis\n58 Direct...,Management Discussion and Analysis \n52 Direc...
5,Welspun India Ltd.,Textiles,WELSPUNIND,EQ,INE192B01031,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,content/WELSPUNIND_2022.pdf,content/WELSPUNIND_2021.pdf,content/WELSPUNIND_2020.pdf,content/WELSPUNIND_2019.pdf,content/WELSPUNIND_2018.pdf,Management Discussion and Analysis\n98 Directo...,Management Discussion and Analysis\n70 Togethe...,Management Discussion and Analysis (MDA) shoul...,Management Discussion and Analysis (MDA) \nsho...,Management Discussion and Analysis\n 54 Direct...
6,Westlife Development Ltd.,Consumer Services,WESTLIFE,EQ,INE274F01020,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bseplus/AnnualReport/...,https://www.bseindia.com/bsepl

In [17]:
data3.to_csv("MDnA_data.csv",index=False)

In [18]:
#